In [1]:
from __future__ import print_function
import pickle
with open('cache/df.pickle', mode='rb') as h:
    _, df_test = pickle.load(h)
with open('cache/hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)

In [2]:
from collections import Counter
labels = df_test['label']
label_counter = Counter(labels)  # dict<label, #_of_occurrence>
id_hls = map(set, hex_data['id_hierarchical_labels'])  # id_hls for id_hierarchical_labels
id_fh = map(lambda x: max(x, key=len), id_hls)  # id_fh for id_full_hierarchy

In [3]:
import numpy as np
def get_accuracy(Y, threshold=0, normalize=True):
    """
    Calculates the accuracy of a prediction, decomposed to labels. Numerical predictions are thresholded.
    Type 0: leaf accuracy, as used in Deng et al, p13. Note that for boolean y from CRF, leaf nodes compete in state space, whereas for numerical y, leaf nodes compete explicitly
    Type 1: prediction IS the full hierarchy
    Type 2: prediction COVERS the full hierarchy
    Type 3: prediction IS any partial hierarchy
    Type 4: prediction COVERS any partial hierarchy
    Implication: 1->2, 1->3, 2->4, 3->4
    Note that type 2 and 3 are mutually exclusive.
    Args:
        Y: N * D matrix of prediction. Data type may be either numerical or boolean.
        threshold: binarize numerical prediction for accuracy type 1-4. Not needed for binary prediction.
        normalize: if False, returns positive count instead of accuracy.
    Returns:
        21 * 5 matrix, where the last row denotes overall accuracy.
    """
    is_bool = Y.dtype == bool
    by_label = np.zeros((20, 5), dtype=int)
    for i in range(0, len(Y)):
        l, y = labels[i], Y[i]
        if (is_bool and y[l]) or (not is_bool and np.argmax(y[:20]) == l):
            by_label[l, 0] += 1
        y = np.nonzero(y > threshold)[0]
        if tuple(y) == id_fh[l]:
            by_label[l, 1:] += 1
        elif set(id_fh[l]) in set(y):
            by_label[l, [2, 4]] += 1
        elif tuple(y) in id_hls[l]:
            by_label[l, [3, 4]] += 1
        elif any(set(x) in set(y) for x in id_hls[l]):
            by_label[l, 4] += 1
    overall = np.sum(by_label, axis=0, dtype=int)
    if normalize:
        by_label = by_label.astype(np.float32)
        for i in range(0, 20):
            by_label[i] /= label_counter[i]
        overall = overall.astype(np.float32) / len(Y)
    return np.vstack((by_label, overall))

In [4]:
np.set_printoptions(threshold=np.nan, precision=4)
id_name = hex_data['id_name']
def accuracy_str(accuracy):
    lines = str(accuracy).splitlines()
    lines = map(lambda x: x.replace(' [', '['), lines)
    lines[0] = lines[0].replace('[[', '[')
    for i in range(0, 20):
        lines[i] = '{} {}'.format(lines[i], id_name[i])
    lines[-1] = '{} {}'.format(lines[-1].replace(']]', ']'), 'overall')
    return '\n'.join(lines)

In [5]:
def opt_model(model_Y, threshold=0):
    print('threshold={0:.2f}'.format(threshold), end='\r')
    scores = [get_accuracy(Y, threshold)[-1, 0] for Y in model_Y]
#     scores = [get_accuracy(Y, threshold)[-1, [0, 1, 3]].prod() for Y in model_Y]
    opt_model = np.argmax(scores)
    return opt_model, scores[opt_model]

In [6]:
def opt_model_threshold(model_Y):
    """
    Performes an exaustive search for the model and threshold, as used in @get_accuracy.
    The range of threshold is [-0.5, 0.5], with step size 0.1.
    Args:
        model_Y: M * N * D matrix, where M is the number of models.
    """
    thresholds = np.linspace(-0.5, 0.5, num=101)
    thres_opt_model, thres_score = zip(*[opt_model(model_Y, t) for t in thresholds])
    
    print(thres_opt_model, thres_score)
    
    opt_thres = np.argmax(thres_score)
    return thres_opt_model[opt_thres], thresholds[opt_thres]

In [7]:
state_space = hex_data['state_space']
def to_crf(Y, leaf_weight=4):
    w = np.hstack(([leaf_weight] * 20, [1] * (len(id_name) - 20)))
    def to_crf_step(y):
        scores = map(lambda s: (w * y)[s].sum(), state_space)
        return state_space[np.argmax(scores)]
    return np.array(map(to_crf_step, Y), dtype=bool)

In [8]:
# caffe
iter_Y = np.load('results/test_caffe_5.npy')
# iter_Y = np.tanh(iter_Y)
opt_iter, _ = opt_model(iter_Y)
print('opt_iter={}'.format(opt_iter))
print('raw\n' + accuracy_str(get_accuracy(iter_Y[opt_iter])))
print('crf\n' + accuracy_str(get_accuracy(to_crf(iter_Y[opt_iter]))))

opt_iter=6
raw
[ 0.3878  0.      0.      0.      0.    ] diningtable
[ 0.037   0.      0.      0.      0.    ] chair
[ 0.3208  0.      0.      0.      0.    ] sofa
[ 0.0345  0.      0.      0.      0.    ] bottle
[ 0.087   0.      0.      0.      0.    ] pottedplant
[ 0.5714  0.      0.      0.      0.    ] tvmonitor
[ 0.7822  0.      0.      0.      0.    ] train
[ 0.6582  0.      0.      0.      0.    ] bus
[ 0.5804  0.      0.      0.      0.    ] car
[ 0.5     0.      0.      0.      0.    ] bicycle
[ 0.6615  0.      0.      0.      0.    ] motorbike
[ 0.7883  0.      0.      0.      0.    ] aeroplane
[ 0.7067  0.      0.      0.      0.    ] boat
[ 0.0426  0.      0.      0.      0.    ] cow
[ 0.169   0.      0.      0.      0.    ] horse
[ 0.      0.      0.      0.      0.    ] sheep
[ 0.3591  0.      0.      0.      0.    ] dog
[ 0.766   0.      0.      0.      0.    ] cat
[ 0.4553  0.      0.      0.      0.    ] bird
[ 0.5412  0.      0.      0.      0.    ] person
[ 0.5107  

In [9]:
# svm
kernel_Y = np.load('results/test_svm_5.npy')
# kernel_Y = np.log(kernel_Y / (1 - kernel_Y))  # log ratio for probability
opt_kernel, _ = opt_model(kernel_Y)
print('opt_kernel={0}'.format(opt_kernel))
print('raw\n' + accuracy_str(get_accuracy(kernel_Y[opt_kernel])))
print('crf\n' + accuracy_str(get_accuracy(to_crf(kernel_Y[opt_kernel]))))

opt_kernel=2
raw
[ 0.6939  0.      0.      0.      0.    ] diningtable
[ 0.4259  0.      0.      0.      0.    ] chair
[ 0.5849  0.      0.      0.      0.    ] sofa
[ 0.6207  0.      0.      0.      0.    ] bottle
[ 0.4783  0.      0.      0.      0.    ] pottedplant
[ 0.8333  0.      0.      0.      0.    ] tvmonitor
[ 0.8317  0.      0.      0.      0.    ] train
[ 0.7848  0.      0.      0.      0.    ] bus
[ 0.7622  0.      0.      0.      0.    ] car
[ 0.7414  0.      0.      0.      0.    ] bicycle
[ 0.7231  0.      0.      0.      0.    ] motorbike
[ 0.8686  0.      0.      0.      0.    ] aeroplane
[ 0.8133  0.      0.      0.      0.    ] boat
[ 0.617   0.      0.      0.      0.    ] cow
[ 0.5915  0.      0.      0.      0.    ] horse
[ 0.5263  0.      0.      0.      0.    ] sheep
[ 0.7514  0.      0.      0.      0.    ] dog
[ 0.8369  0.      0.      0.      0.    ] cat
[ 0.8293  0.      0.      0.      0.    ] bird
[ 0.7732  0.      0.      0.      0.    ] person
[ 0.7506